# Pipeline

Funciones utilizadas como pipeline. Se usan resultados del notebook de data_collection y EDA, especialmente para preprocesar el conjunto de test.

In [1]:
import pandas as pd
import numpy as np
import random

import os
import glob
import pickle

import geopandas as gpd

from scipy import stats
from sklearn.preprocessing import StandardScaler
import joblib

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor


In [2]:
# Auxiliary functions

def get_q1_q3(df):
    """
    Used as an auxiliary function of 'get_outliers'.
    Create new columns with the values of the first quartile (q1) and the third one (q3).
    Also adds the IQR to ease the calculation of the following variables.
    """
    temp = df.loc[:, ["Price", "Year", "month", "day"]].\
        groupby(["Year", "month", "day"]).agg(
            q1=("Price", lambda x: np.percentile(x, 25)),
            q3=("Price", lambda x: np.percentile(x, 75))
        ).reset_index()
    
    temp["IQR"] = temp["q3"] - temp["q1"]
    
    return pd.merge(left=df, right=temp, how="left", on=("Year", "month", "day"))

def get_outliers(df):
    """
    Creates the new variables from 'get_q1_q3' and another two, which indicate if the price value
    is above the expected value for that district on the specified time or below.
    """
    temp = df.copy()
    temp = get_q1_q3(temp)
    temp.loc[:, "is_top_outlier"] = (temp["Price"] > (temp["q3"] + 1.5 * temp["IQR"])).astype("int")
    temp.loc[:, "is_bottom_outlier"] = (temp["Price"] < (temp["q1"] - 1.5 * temp["IQR"])).astype("int")
    temp.drop(["IQR"], axis=1, inplace=True) # b/c is linear combination of q1 and q3
    return temp

def create_mean_price(df):
    if "mean_price" not in df.columns:
        temp = df.groupby(["Year", "month", "day"])["Price"].\
            mean("Price").rename("mean_price")
        return pd.merge(
            left=df, right=temp,
            how="left", on=["Year", "month", "day"]
        )
    return df

def inv_boxcox(y, lambda_param):
    if lambda_param == 0:
        return np.exp(y)
    else:
        return np.exp(np.log(lambda_param * y + 1) / lambda_param)

def create_year_to_year_change(df):
    temp = df.groupby(["District", "Year", "Property.Type"]).mean("Price").diff()
    temp = temp.groupby(["District", "Property.Type"])["Price"].pct_change().reset_index()

    temp = temp.rename(columns={"Price": "year_to_year_change"}) # rename column
    temp = temp.replace([np.inf, -np.inf], np.nan)
    temp = temp.fillna(value={
        "year_to_year_change": np.mean(temp["year_to_year_change"])}) 
    
    return pd.merge(left=df, right=temp, how="left", on=("District", "Year", "Property.Type"))

def create_price_comparison_london(df):
    reference_district = "city of london"
    df["price_compare_london"] = df["Price"]

    temp = df.groupby(["District", "Year"])["Price"].mean("Price").reset_index()
    temp = temp.loc[temp.District == reference_district].set_index("Year")

    for year in temp.index:
        df.loc[df["Year"] == year, "price_compare_london"] = df.loc[df["Year"] == year, "price_compare_london"] / temp.loc[year, "Price"]

def variable_modification(df):
    x_names = ["price_compare_london",
               "q1", "q3", "mean_price",
               "year_to_year_change"]
    
    for x in [v for v in x_names if "log_"+v not in df.columns]:
        newname = "log_"+x
        df[newname] = np.log(np.abs(df[x]) + 0.01)

def create_year_to_year_change_test(df, train_df):
    temp = pd.concat([train_df.loc[:, ["Year", "mean_price"]].copy(),
                      df.loc[:, ["Year", "mean_price"]]])
    temp = temp.groupby(["Year"]).mean("mean_price").diff().iloc[-4:, :]
    temp = temp["mean_price"].pct_change().reset_index().iloc[-3:, :]

    temp = temp.rename(columns={"mean_price": "year_to_year_change"}) # rename column    
    return pd.merge(left=df, right=temp, how="left", on=("Year"))

def create_price_compare_london_test(df, train_df=None):
    temp = train_df.\
        loc[:,["District", "price_compare_london"]].\
            drop_duplicates()
    temp = temp.groupby("District").agg(np.mean).reset_index()

    df = pd.merge(
        left=df, right=temp,
        how="left", on="District"
    )

    df["price_compare_london"] = df["price_compare_london"].\
        fillna(value=np.mean(df["price_compare_london"])) 
    return df

In [3]:
def pipeline(df, is_train_data, lambda_boxcox=None, train_df=None):
    # --- Initialization ---
    # Drop columns and filter cases
    unusable_columns = ["Transaction.unique.identifier", 
                        "Record.Status...monthly.file.only",
                        "PPD.Category.Type"]
    discarded_columns = ["PAON", "SAON", "Postcode", 
                         "Locality", "Street"]
    if "Unnamed: 0" in df.columns:
         unusable_columns = ["Unnamed: 0"] + unusable_columns
    to_drop = unusable_columns + discarded_columns
    df = df.drop(to_drop, axis=1)

    mask = (df["Property.Type"] != "Otro") &\
        (df["Duration"] != "Desconocido")
    df = df.loc[mask, :]

    # Encoding
    mapping = {
        "Duration": {"Propiedad": 0, "Alquiler": 1, "Desconocido": 2},
        "Property.Type": {"Adosado": 0, "Semi-adosado": 1, "Unifamiliar": 2, 
                          "Piso/Apartamento": 3, "Otro": 4}
    }
    df["org_property_type"] = df["Property.Type"]
    df["org_duration"] = df["Duration"]
    df = df.replace(mapping)

    df["First_hand"] = (df["Old.New"] == "Nuevo").astype("int") # Binary

    # Date.of.Transfer to year, month and day
    df["Date.of.Transfer"] = pd.to_datetime(df["Date.of.Transfer"])
    df["Year"] = df["Date.of.Transfer"].dt.year
    df["month"] = df["Date.of.Transfer"].dt.month
    df["day"] = df["Date.of.Transfer"].dt.day

    # Lower case
    to_lower_cols = ("Town.City","District","County")
    for col in to_lower_cols:
        df.loc[:, col] = df.loc[:, col].str.lower()

    # Correct district name
    # In the data exists the wrekin and wrekin separatedly
    # Maybe b/c they are different region over time
    df.loc[:, "District"] = df.loc[:, "District"].replace(to_replace="the wrekin", value="wrekin")

    # --- Enrichment data ---
    # Load census data
    with open('../output/census_df.pkl', 'rb') as f:
                census_df = pickle.load(f)
    # census_df = census_df.drop(["BandG", "total_houses",
    #                             "people_studying", "people_working"], 
    #                             axis=1)

    # Load the UK regions shapefile
    with open('../output/uk_geo.pkl', 'rb') as f:
        uk_geo = pickle.load(f)
    uk_geo = uk_geo.loc[:, ["District", "is_coastal", "is_isle"]]

    # Lat/Long of Towns/Cities
    with open('../output/town_city_coords_final.pkl', 'rb') as f:
        town_city_coords = pickle.load(f)

    town_city_coords = town_city_coords.loc[:, ["County", "District", 
                                                "Town.City", "spatial_cluster",
                                                "distance_to_london"]]

    # --- Train pipeline ---
    if is_train_data:
        # --- Outliers ---
        df = get_outliers(df)
    
        # --- New vars from original data ---
        df = create_mean_price(df)

        df = create_year_to_year_change(df)

        create_price_comparison_london(df) # is inplace

        # district_effects
        with open('../output/district_effects.pkl', 'rb') as f:
            district_effects = pickle.load(f)
        df["fixed_effect"] = pd.merge(
             left=df.loc[:, "District"], 
             right=district_effects,
             left_on="District", right_on="index", 
             how="left").\
                rename(columns={"fixed_effect": "district_effects"}).\
                    loc[:, "district_effects"]
        del district_effects

        df["Price_boxcox"] = stats.boxcox(df["Price"], lambda_boxcox)
        # df.drop("Price", axis=1, inplace=True)

        # --- Join enrichment data ---
        # Census data
        df = pd.merge(
             left=df, right=census_df,
             how="left", on="District"
        )
        census_numeric_columns = census_df.select_dtypes(include=['number']).columns
        df[census_numeric_columns] = df[census_numeric_columns].fillna(df[census_numeric_columns].mean())
        del census_df

        # Maps data
        df = pd.merge(
            left = df, right=uk_geo,
            how="left", on="District"
        )
        del uk_geo
        df.loc[df["is_isle"].isna(), ["is_isle"]] = 0
        df.loc[df["is_coastal"].isna(), ["is_coastal"]] = 0


        # Towns/Cities position data
        df = pd.merge(left=df, right=town_city_coords, 
                      on=["County", "District", "Town.City"], 
                      how="left")
        del town_city_coords

        # --- Modificación de variables ---
        variable_modification(df)

        # return df 
    
        # --- Escalado ---
        scaler = joblib.load('../output/scaler.gz')
        to_scale = scaler.feature_names_in_

        df["org_mean_price"] = df["mean_price"]
        df["org_price_boxcox"] = df["Price_boxcox"]
        df[to_scale] = scaler.transform(df[to_scale])


        return df
    
    else:
        # Test data set
        # --- Outliers ---
        # log_q1, log_q3, log_mean_price
        with open('../output/q1_q3_test.pkl', 'rb') as f:
            q1_q3_test = pickle.load(f)
        df = pd.merge(left=df, right=q1_q3_test, 
                      on=["Year", "month"], how="left")
        
        # q1, q3, mean_price
        vars_forecast = ["q1", "q3", "mean_price"]
        log_vars_forecast = ['log_q1', 'log_q3', 'log_mean_price']
        for v, v_log in zip(vars_forecast, log_vars_forecast):
            df[v] = np.exp(df[v_log])

        # is_top_outlier, is_bottom_outlier
        df["IQR"] = df["q3"] - df["q1"]

        df.loc[:, "is_top_outlier"] = (df["Price"] > (df["q3"] + 1.5 * df["IQR"])).astype("int")
        df.loc[:, "is_bottom_outlier"] = (df["Price"] < (df["q1"] - 1.5 * df["IQR"])).astype("int")
        df.drop(["IQR"], axis=1, inplace=True) # b/c is linear combination of q1 and q3


        # --- New vars from original data ---
        df = create_year_to_year_change_test(df, train_df)

        df = create_price_compare_london_test(df, train_df)

        with open('../output/district_effects.pkl', 'rb') as f:
            district_effects = pickle.load(f)
        df["fixed_effect"] = pd.merge(
            left=df.loc[:, "District"], 
            right=district_effects,
            left_on="District", right_on="index", 
            how="left").\
                rename(columns={"fixed_effect": "district_effects"}).\
                    loc[:, "district_effects"]
        del district_effects

        df["Price_boxcox"] = stats.boxcox(df["Price"], lambda_boxcox)
        # df.drop("Price", axis=1, inplace=True)

        # --- Join enrichment data ---
        # Census data
        df = pd.merge(
            left=df, right=census_df,
            how="left", on="District"
        )
        census_numeric_columns = census_df.select_dtypes(include=['number']).columns
        df[census_numeric_columns] = df[census_numeric_columns].fillna(df[census_numeric_columns].mean())
        del census_df

        # Maps data
        df = pd.merge(
            left = df, right=uk_geo,
            how="left", on="District"
        )
        del uk_geo
        df.loc[df["is_isle"].isna(), ["is_isle"]] = 0
        df.loc[df["is_coastal"].isna(), ["is_coastal"]] = 0

        # Towns/Cities position data
        df = pd.merge(left=df, right=town_city_coords, 
                    on=["County", "District", "Town.City"], 
                    how="left")
        del town_city_coords

        # --- Modificación de variables ---
        variable_modification(df)

        # --- Escalado ---
        scaler = joblib.load('../output/scaler.gz')
        to_scale = scaler.feature_names_in_

        df["org_mean_price"] = df["mean_price"]
        df["org_price_boxcox"] = df["Price_boxcox"]
        df[to_scale] = scaler.transform(df[to_scale])


        return df
    
   



In [4]:
import funciones_auxiliares as fun
import pyarrow as pa
import pyarrow.parquet as pq

with open('../output/lambda_boxcox.pkl', 'rb') as f:
                lambda_boxcox = pickle.load(f)
                
columns_dtype = {'Transaction unique identifier' : np.dtype(str),
                 'Price' : np.dtype(int),
                 'Date of Transfer' : np.dtype(str),
                 'Postcode' : np.dtype(str),
                 'Property Type' : np.dtype(str),
                 'Old/New' : np.dtype(str),
                 'Duration' : np.dtype(str),
                 'PAON' : np.dtype(str),
                 'SAON' : np.dtype(str),
                 'Street' : np.dtype(str),
                 'Locality' : np.dtype(str),
                 'Town/City' : np.dtype(str),
                 'District' : np.dtype(str),
                 'County' : np.dtype(str),
                 'PPD Category Type' : np.dtype(str),
                 'Record Status - monthly file only' : np.dtype(str)}

## Parquet data

In [5]:
# --- Train data set ---
parquet_file = '../data/parquet/train/train.parquet'
pd.options.mode.chained_assignment = None  # default='warn'

for i,year in enumerate(np.arange(1995, 2021, 1)):
    print(f"\rAño: {year}", end="")
    chunk = fun.load_date(year, columns_dtype=columns_dtype)
    chunk = pipeline(chunk, is_train_data=True, lambda_boxcox=lambda_boxcox)
    
    if i == 0:
        # Guess the schema of the CSV file from the first chunk
        parquet_schema = pa.Table.from_pandas(df=chunk).schema
        # Open a Parquet file for writing
        parquet_writer = pq.ParquetWriter(parquet_file, parquet_schema, compression='snappy')
    # Write CSV chunk to the parquet file
    table = pa.Table.from_pandas(chunk, schema=parquet_schema)
    parquet_writer.write_table(table)

parquet_writer.close()

Año: 2020

In [6]:
# --- Test data set ---
parquet_file = '../data/parquet/test/test.parquet'
pd.options.mode.chained_assignment = None  # default='warn'

with open('../output/train_df_pre_escalado.pkl', 'rb') as f:
            train_df = pickle.load(f) # Sample

for i,year in enumerate(np.arange(2021, 2024, 1)):
    print(f"\rAño: {year}", end="")
    chunk = fun.load_date(year, columns_dtype=columns_dtype)
    chunk = pipeline(chunk, is_train_data=False, 
                     lambda_boxcox=lambda_boxcox,
                     train_df=train_df)
    if i == 0:
        # Guess the schema of the CSV file from the first chunk
        parquet_schema = pa.Table.from_pandas(df=chunk).schema
        # Open a Parquet file for writing
        parquet_writer = pq.ParquetWriter(parquet_file, parquet_schema, compression='snappy')
    # Write CSV chunk to the parquet file
    table = pa.Table.from_pandas(chunk, schema=parquet_schema)
    parquet_writer.write_table(table)

parquet_writer.close()

Año: 2023

## Muestras

In [7]:
# https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

# Train data set
path = "../data/train/"
all_files = glob.glob(os.path.join(path, "*.csv"))
train_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# Test data set
path = "../data/test/"
all_files = glob.glob(os.path.join(path, "*.csv"))
test_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

train_df.head(1)

,Unnamed: 0,Transaction.unique.identifier,Price,Date.of.Transfer,Postcode,Property.Type,Old.New,Duration,PAON,SAON,Street,Locality,Town.City,District,County,PPD.Category.Type,Record.Status...monthly.file.only
0,0,{677AA6E5-A626-461A-BE2B-7B0580842D46},300000,1995-01-01 00:00,SW19 5JU,Piso/Apartamento,Nuevo,Alquiler,1,FLAT 6,SOMERSET ROAD,LONDON,LONDON,MERTON,GREATER LONDON,A,A


In [8]:
pd.options.mode.chained_assignment = None  # default='warn'

temp = pipeline(train_df, is_train_data=True, 
                    lambda_boxcox=lambda_boxcox)

temp.head()

,Price,Date.of.Transfer,Property.Type,Old.New,Duration,Town.City,District,County,org_property_type,org_duration,...,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_q1,log_q3,log_mean_price,log_year_to_year_change,org_mean_price,org_price_boxcox
0,300000,1995-01-01,3,Nuevo,1,london,merton,greater london,Piso/Apartamento,Alquiler,...,-0.057229,1.687222,-1.338552,3.112323,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,14.442841
1,70500,1995-01-01,1,Segunda_mano,0,horsham,horsham,west sussex,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.996887,1.007904,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,12.583137
2,35000,1995-01-01,0,Segunda_mano,0,dewsbury,kirklees,west yorkshire,Adosado,Propiedad,...,-0.057229,0.848829,0.928138,0.003143,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.703994
3,41000,1995-01-01,1,Segunda_mano,0,gloucester,gloucester,gloucestershire,Semi-adosado,Propiedad,...,-0.057229,-0.827957,-0.072118,0.228881,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,11.901504
4,125000,1995-01-01,2,Segunda_mano,0,macclesfield,macclesfield,cheshire,Unifamiliar,Propiedad,...,-0.057229,-0.827957,0.698371,1.837439,-1.383724,-0.673885,-0.848781,-0.732369,112464.285714,13.311820


In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(temp.describe())

,Price,Property.Type,Duration,First_hand,Year,month,day,q1,q3,is_top_outlier,is_bottom_outlier,mean_price,year_to_year_change,price_compare_london,fixed_effect,Price_boxcox,total_people,people_working,people_studying,total_houses,BandA,BandB,BandC,BandD,BandE,BandF,BandG,job_density,density_population_hectare,is_coastal,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_q1,log_q3,log_mean_price,log_year_to_year_change,org_mean_price,org_price_boxcox
count,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06
mean,1.899498e+05,1.294830e+00,2.364898e-01,1.071355e-01,2.006870e+03,6.827016e+00,1.707934e+01,-6.688110e-03,1.599919e-02,6.180764e-02,3.116090e-05,1.684655e-02,-4.862383e-03,-1.068375e-02,1.261908e-04,5.807212e-03,-6.616829e-02,1.203556e+05,1.203556e+05,5.543102e+04,-6.536091e-03,-2.309007e-03,8.300798e-03,5.581900e-03,-6.088127e-03,-1.539368e-02,6.582191e-01,-7.695388e-03,4.999529e-03,-8.978798e-04,-5.120508e-04,1.160899e-03,1.538757e-03,-2.653100e-02,3.678746e-03,1.753269e-02,1.790441e-02,2.217523e-03,1.899498e+05,1.343414e+01
std,2.343114e+05,1.088735e+00,4.249264e-01,3.092854e-01,7.434252e+00,3.340135e+00,9.036621e+00,9.664788e-01,9.916547e-01,2.408059e-01,5.582109e-03,9.944389e-01,1.384902e+00,1.005555e+00,1.009998e+00,1.001059e+00,9.789747e-01,8.339029e+04,8.339029e+04,3.965486e+04,9.975222e-01,1.001522e+00,1.000752e+00,1.001608e+00,1.000859e+00,9.973864e-01,5.879786e-01,1.003991e+00,1.003386e+00,9.995264e-01,9.955310e-01,1.000308e+00,1.001115e+00,1.018887e+00,9.815505e-01,9.948757e-01,9.954686e-01,1.000463e+00,7.966142e+04,1.027701e+00
min,1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.995000e+03,1.000000e+00,1.000000e+00,-2.413638e+00,-2.357472e+00,0.000000e+00,0.000000e+00,-2.348116e+00,-2.080607e+02,-9.843580e-01,-4.796309e+01,-8.369797e+00,-1.451265e+00,1.720000e+04,1.720000e+04,5.650000e+02,-9.951011e-01,-1.927145e+00,-5.050178e+00,-4.467860e+00,-2.914834e+00,-1.435804e+00,1.000000e-01,-2.514293e-01,-7.938802e-01,-6.033085e-01,-5.722919e-02,-8.279566e-01,-1.472863e+00,-5.250589e+00,-1.088002e+01,-1.241139e+01,-1.203792e+01,-2.420810e+00,5.000000e+02,4.835626e+00
25%,8.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,4.000000e+00,9.000000e+00,-1.062268e+00,-8.981725e-01,0.000000e+00,0.000000e+00,-9.321656e-01,-8.274375e-03,-5.130529e-01,-4.100532e-02,-6.671920e-01,-6.670355e-01,6.410000e+04,6.410000e+04,3.028200e+04,-6.741107e-01,-7.487002e-01,-5.970111e-01,-5.693987e-01,-6.258460e-01,-6.568843e-01,4.000000e-01,-1.071848e-01,-6.732381e-01,-6.033085e-01,-5.722919e-02,-8.279566e-01,-9.071521e-01,-6.798812e-01,-9.881369e-01,-7.736013e-01,-8.220762e-01,-6.698531e-01,1.139274e+05,1.274323e+01
50%,1.400000e+05,1.000000e+00,0.000000e+00,0.000000e+00,2.006000e+03,7.000000e+00,1.800000e+01,3.752838e-01,1.415119e-01,0.000000e+00,0.000000e+00,1.252817e-01,-2.449885e-03,-2.340147e-01,-4.100463e-02,2.813472e-02,-2.131885e-01,1.055000e+05,1.055000e+05,4.713700e+04,-3.531202e-01,-2.309007e-03,8.300798e-03,5.581900e-03,-6.088127e-03,-1.612081e-01,5.000000e-01,-4.061045e-02,-3.354401e-01,-6.033085e-01,-5.722919e-02,-6.882244e-01,-2.148693e-02,-2.332027e-02,5.159193e-01,3.385457e-01,3.264587e-01,-3.693272e-02,1.986362e+05,1.345706e+01
75%,2.300000e+05,2.000000e+00,0.000000e+00,0.000000e+00,2.014000e+03,1.000000e+01,2.500000e+01,6.924789e-01,7.909270e-01,0.000000e+00,0.000000e+00,8.160821e-01,4.612903e-03,1.912052e-01,-4.095034e-02,6.518504e-01,1.600635e-01,1.445000e+05,1.445000e+05,6.378600e+04,-6.536091e-03,3.917305e-01,5.817683

In [10]:
with open('../output/train_df_pre_escalado.pkl', 'rb') as f:
            train_df = pickle.load(f) # Sample

temp_test = pipeline(test_df, is_train_data=False, 
                   lambda_boxcox=lambda_boxcox,
                   train_df=train_df)

temp_test.head()

            
with open('../output/test_df_pipeline.pkl', 'wb') as f:
            pickle.dump(temp_test, f)


In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(temp_test.describe())

,Price,Property.Type,Duration,First_hand,Year,month,day,log_q1,log_q3,log_mean_price,q1,q3,mean_price,is_top_outlier,is_bottom_outlier,year_to_year_change,price_compare_london,fixed_effect,Price_boxcox,total_people,people_working,people_studying,total_houses,BandA,BandB,BandC,BandD,BandE,BandF,BandG,job_density,density_population_hectare,is_coastal,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_year_to_year_change,org_mean_price,org_price_boxcox
count,2.097850e+05,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.0,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000,209785.000000
mean,3.471065e+05,1.295989,0.226827,0.064385,2021.508263,6.051062,17.469562,1.019191,0.877089,0.915793,1.140624,0.892635,0.947944,0.142670,0.0,-0.069934,-0.000010,0.003262,0.859399,-0.006663,124255.595388,124255.595388,57680.833513,0.019646,0.045372,-0.037463,-0.051000,0.013148,0.061104,0.694944,-0.021318,-0.059806,-0.001957,-0.000739,-0.016278,0.062488,0.246860,1.522766,264537.202807,14.310446
std,3.878037e+05,1.079481,0.418781,0.245438,0.606250,3.381460,8.971636,0.073491,0.149084,0.104157,0.135961,0.231246,0.163833,0.349737,0.0,0.033950,0.484183,1.565745,0.860614,1.001093,85500.460600,85500.460600,40684.211499,1.010407,1.030592,1.007282,1.036703,1.021529,1.072508,0.642602,0.830725,0.944288,0.998969,0.993548,0.988485,1.000863,0.551790,0.487605,13124.127336,0.883519
min,1.000000e+00,0.000000,0.000000,0.000000,2021.000000,1.000000,1.000000,0.957585,0.479163,0.692884,1.034728,0.317254,0.606474,0.000000,0.0,-0.097797,-0.590962,-47.963092,-13.080064,-1.451265,17200.000000,17200.000000,565.000000,-0.995101,-1.927145,-5.050178,-4.467860,-2.914834,-1.435804,0.100000,-0.251429,-0.793880,-0.603309,-0.057229,-0.827957,-1.472863,-0.931673,0.914039,237183.094159,0.000000
25%,1.740000e+05,0.000000,0.000000,0.000000,2021.000000,3.000000,10.000000,1.002659,0.776579,0.853454,1.110204,0.730648,0.845765,0.000000,0.0,-0.097797,-0.291900,-0.041005,0.300487,-0.675609,63700.000000,63700.000000,30104.000000,-0.674111,-0.748700,-0.706157,-0.769891,-0.657638,-0.586073,0.400000,-0.107185,-0.673238,-0.603309,-0.057229,-0.827957,-0.807224,-0.137407,0.914039,256351.958018,13.736660
50%,2.699500e+05,1.000000,0.000000,0.000000,2021.000000,6.000000,18.000000,1.003715,0.879654,0.946301,1.111992,0.888328,0.992859,0.000000,0.0,-0.097797,-0.139518,-0.041005,0.854464,-0.249771,102200.000000,102200.000000,45123.000000,-0.353120,-0.026427,-0.037463,-0.012476,0.009984,-0.161208,0.500000,-0.046158,-0.371633,-0.603309,-0.057229,-0.688224,0.030570,0.145706,1.918864,268135.213385,14.305380
75%,4.100000e+05,2.000000,0.000000,0.000000,2022.000000,9.000000,26.000000,1.020429,0.952901,0.984993,1.140410,1.005237,1.056136,0.000000,0.0,-0.027587,0.139828,-0.040946,1.386395,0.339544,160800.000000,160800.000000,73559.000000,0.288861,0.657831,0.581768,0.678109,0.486857,0.334468,0.800000,0.003772,0.102893,1.657527,-0.057229,1.128294,0.922769,0.554745,1.918864,273204.058599,14.851468
max,4.576436e+07,3.000000,1.000000,1.000000,2023.000000,12.000000,31.000000,1.415391,1.158511,1.076601,1.882592,1.356368,1.210759,1.000000,0.0,-0.027587,4.376268,151.234615,7.723321,5.200965,540300.000000,540300.000000,252064.000000,7.350652,4.592317,3.135791,2.616201,5.382748,12.513942,10.400000,56.630834,4.775765,1.657527,17.473599,1.826955,2.984267,2.819267,1.918864,285590.464866,21.357045


In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(temp.describe())

,Price,Property.Type,Duration,First_hand,Year,month,day,q1,q3,is_top_outlier,is_bottom_outlier,mean_price,year_to_year_change,price_compare_london,fixed_effect,Price_boxcox,total_people,people_working,people_studying,total_houses,BandA,BandB,BandC,BandD,BandE,BandF,BandG,job_density,density_population_hectare,is_coastal,is_isle,spatial_cluster,distance_to_london,log_price_compare_london,log_q1,log_q3,log_mean_price,log_year_to_year_change,org_mean_price,org_price_boxcox
count,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06,2.567320e+06
mean,1.899498e+05,1.294830e+00,2.364898e-01,1.071355e-01,2.006870e+03,6.827016e+00,1.707934e+01,-6.688110e-03,1.599919e-02,6.180764e-02,3.116090e-05,1.684655e-02,-4.862383e-03,-1.068375e-02,1.261908e-04,5.807212e-03,-6.616829e-02,1.203556e+05,1.203556e+05,5.543102e+04,-6.536091e-03,-2.309007e-03,8.300798e-03,5.581900e-03,-6.088127e-03,-1.539368e-02,6.582191e-01,-7.695388e-03,4.999529e-03,-8.978798e-04,-5.120508e-04,1.160899e-03,1.538757e-03,-2.653100e-02,3.678746e-03,1.753269e-02,1.790441e-02,2.217523e-03,1.899498e+05,1.343414e+01
std,2.343114e+05,1.088735e+00,4.249264e-01,3.092854e-01,7.434252e+00,3.340135e+00,9.036621e+00,9.664788e-01,9.916547e-01,2.408059e-01,5.582109e-03,9.944389e-01,1.384902e+00,1.005555e+00,1.009998e+00,1.001059e+00,9.789747e-01,8.339029e+04,8.339029e+04,3.965486e+04,9.975222e-01,1.001522e+00,1.000752e+00,1.001608e+00,1.000859e+00,9.973864e-01,5.879786e-01,1.003991e+00,1.003386e+00,9.995264e-01,9.955310e-01,1.000308e+00,1.001115e+00,1.018887e+00,9.815505e-01,9.948757e-01,9.954686e-01,1.000463e+00,7.966142e+04,1.027701e+00
min,1.000000e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.995000e+03,1.000000e+00,1.000000e+00,-2.413638e+00,-2.357472e+00,0.000000e+00,0.000000e+00,-2.348116e+00,-2.080607e+02,-9.843580e-01,-4.796309e+01,-8.369797e+00,-1.451265e+00,1.720000e+04,1.720000e+04,5.650000e+02,-9.951011e-01,-1.927145e+00,-5.050178e+00,-4.467860e+00,-2.914834e+00,-1.435804e+00,1.000000e-01,-2.514293e-01,-7.938802e-01,-6.033085e-01,-5.722919e-02,-8.279566e-01,-1.472863e+00,-5.250589e+00,-1.088002e+01,-1.241139e+01,-1.203792e+01,-2.420810e+00,5.000000e+02,4.835626e+00
25%,8.000000e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.001000e+03,4.000000e+00,9.000000e+00,-1.062268e+00,-8.981725e-01,0.000000e+00,0.000000e+00,-9.321656e-01,-8.274375e-03,-5.130529e-01,-4.100532e-02,-6.671920e-01,-6.670355e-01,6.410000e+04,6.410000e+04,3.028200e+04,-6.741107e-01,-7.487002e-01,-5.970111e-01,-5.693987e-01,-6.258460e-01,-6.568843e-01,4.000000e-01,-1.071848e-01,-6.732381e-01,-6.033085e-01,-5.722919e-02,-8.279566e-01,-9.071521e-01,-6.798812e-01,-9.881369e-01,-7.736013e-01,-8.220762e-01,-6.698531e-01,1.139274e+05,1.274323e+01
50%,1.400000e+05,1.000000e+00,0.000000e+00,0.000000e+00,2.006000e+03,7.000000e+00,1.800000e+01,3.752838e-01,1.415119e-01,0.000000e+00,0.000000e+00,1.252817e-01,-2.449885e-03,-2.340147e-01,-4.100463e-02,2.813472e-02,-2.131885e-01,1.055000e+05,1.055000e+05,4.713700e+04,-3.531202e-01,-2.309007e-03,8.300798e-03,5.581900e-03,-6.088127e-03,-1.612081e-01,5.000000e-01,-4.061045e-02,-3.354401e-01,-6.033085e-01,-5.722919e-02,-6.882244e-01,-2.148693e-02,-2.332027e-02,5.159193e-01,3.385457e-01,3.264587e-01,-3.693272e-02,1.986362e+05,1.345706e+01
75%,2.300000e+05,2.000000e+00,0.000000e+00,0.000000e+00,2.014000e+03,1.000000e+01,2.500000e+01,6.924789e-01,7.909270e-01,0.000000e+00,0.000000e+00,8.160821e-01,4.612903e-03,1.912052e-01,-4.095034e-02,6.518504e-01,1.600635e-01,1.445000e+05,1.445000e+05,6.378600e+04,-6.536091e-03,3.917305e-01,5.817683

In [13]:
# Save results
with open('../output/train_df_pipeline.pkl', 'wb') as f:
            pickle.dump(temp, f)
